In [11]:
import requests
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
from collections import defaultdict

import re
import os

In [12]:
# taken directly from Dr. Crane's 'TreebankCount.ipynb'
tenses = {'p':'present','f':'future', 'i':'imperfect','a':'aorist','r':'perfect','l':'pluperfect','t':'future perfect'}
voices = {'a':'active','m':'middle','p':'passive', 'i':'imperfect','e':'middle-passive'}
moods = {'i':'indicative','s':'subjunctive', 'o':'optative','m':'imperative','p':'participle','n':'infinitive'}

authinfo = {}
authinfo['tlg0011'] = 'Sophocles,-497,-406,poetry,drama'
authinfo['tlg1220'] = 'Batrachomyomachia,-100,-1,poetry,hexameter'
authinfo['tlg0013'] = 'Homeric Hymns,-650,-450,poetry,hexameter'
authinfo['tlg0020'] = 'Hesiod,-750,-650,poetry,hexameter'
authinfo['tlg0026'] = 'Aeschines,-389,-314,prose,orator'
authinfo['tlg0058'] = 'Aeneas Tacticus,301,400,prose,misc'
authinfo['tlg0096'] = 'Aesop,-620,-564,prose'
authinfo['tlg0085'] = 'Aeschylus,-525c,-455c,poetry,drama'
authinfo['tlg0028'] = 'Antiphon,-490,-411,prose,orator'
authinfo['tlg0551'] = 'Appian,-490,-411,prose,history'
authinfo['tlg0086'] = 'Aristotle,-384,-322,prose,philosophy'
authinfo['tlg0019'] = 'Aristophanes,-446,-386,poetry,drama'
authinfo['tlg0008'] = 'Athenaeus,170,223,prose,misc'
authinfo['tlg0554'] = 'Chariton,101,200,prose,misc'
authinfo['tlg0041'] = 'Chionis Epistulae,1,200,prose,misc'
authinfo['tlg0627'] = 'Corpus Hippocraticum,-450,-350,prose,misc'
authinfo['tlg0014'] = 'Demosthenes,-384,-322,prose,orator'
authinfo['tlg0060'] = 'Diodorus Siculus,-90c,-30c,prose,history'
authinfo['tlg0081'] = 'Dionysius of Halicarnassus,-60c,-7c,prose,history'
authinfo['tlg0557'] = 'Epictetus,50,135,prose,philosopy'
authinfo['tlg0537'] = 'Epicurus,-341,-270,prose,philosophy'
authinfo['tlg0343'] = 'Ezechiel the Tragic Poet,-200,-101,poetry'
authinfo['tlg0006'] = 'Euripides,-480c,-406c,poetry,drama'
authinfo['tlg0016'] = 'Herodotus,-484c,-425c,prose,history'
authinfo['tlg0559'] = 'Heron of Alexandria,10,70,prose,misc'
authinfo['tlg0010'] = 'Isocrates,-436,-338,prose,orator'
authinfo['tlg0526'] = 'Josephus,37,100c,prose,history'
authinfo['tlg2003'] = 'Julian the Apostate,331,363,prose,misc'
authinfo['tlg0561'] = 'Longus,101,200,prose,misc'
authinfo['tlg0061'] = 'Pseudo-Lucian,201,400,prose,misc'
authinfo['tlg0062'] = 'Lucian,125,180,prose,misc'
authinfo['tlg0540'] = 'Lysias,-445c,-380c,prose,orator'
authinfo['tlg0541'] = 'Menander,-342,-291,poetry,drama'
authinfo['tlg0255'] = 'Mimnermus,-650,-600,poetry,lyreleg'
authinfo['tlgX208'] = 'Paeanius,301,400,prose,misc'
authinfo['tlg0585'] = 'Phlegon,151,200,prose,misc'
authinfo['tlg0059'] = 'Plato,-428c,-347c,prose,philosophy'
authinfo['tlg0007'] = 'Plutarch,46,119,prose,history'
authinfo['tlg0543'] = 'Polybius,-200,-118,prose,history'
authinfo['tlg4029'] = 'Procopius,500,565,prose,history'
authinfo['tlg0009'] = 'Sappho,-630,-570,poetry,lyreleg'
authinfo['tlg0260'] = 'Semonides,-700,-601,poetry,lyreleg'
authinfo['tlg0527'] = 'Septuagint,-250,-100,prose,bible'
authinfo['tlg0544'] = 'Sextus Empiricus,150,250,prose,philosophy'
authinfo['tlg0032'] = 'Xenophon,-430c,-354,prose,history'
authinfo['tlg0005'] = 'Theocritus,-300,-255,poetry,hexameter'
authinfo['tlg0003'] = 'Thucydides,-460c,-400c,prose,history'
authinfo['tlg0093'] = 'Theophrastus,-371,-287,prose,philosophy'
authinfo['tlg0012'] = 'Homer,-775c,-700c,poetry,hexameter'

In [13]:
# change ROOT to local root
# ROOT = "c:\\Users\\bella\\OneDrive\\Documents"
ROOT = "/Users/bellahwang/Documents"

LOCALPATH = os.path.join(ROOT, 'GitHub')
GORPATH = os.path.join(LOCALPATH, 'gorman-trees', 'public', 'xml')
PEDPATH = os.path.join(LOCALPATH, 'pedalion-trees', 'public', 'xml')
MAMPATH = os.path.join(LOCALPATH, 'gAGDT', 'data', 'xml')

In [32]:
# taken directly from Dr. Crane's 'TreebankCount.ipynb'
def addfiles(dirname,flist):
    for foo in sorted(os.listdir(dirname)):
        if(not re.search('\.xml$',foo)):
            continue
        newpath = os.path.join(dirname, foo)
        flist.append(newpath)
    return(flist)

In [83]:
#FILENAME = os.path.join(PEDPATH, 'chion.xml')
#FILENAME = os.path.join(GORPATH, 'aeschines-1-1-50-bu1.xml')
#FILENAME = os.path.join(MAMPATH, 'tlg0003.tlg001.perseus-grc1.1.tb.xml')

#list of exceptions
Except = ['""', '_', '-', 'c']

def VerbTenses(FILENAME):
    verbForms = defaultdict(int)
    PrevAuthor = None
    
    with open(FILENAME, 'r', encoding="utf-8") as f:
        soup = BeautifulSoup(f, "xml")
        if soup.treebank.sentence.has_attr('Author'):
            Author = soup.treebank.sentence['Author']
            if not Author == PrevAuthor:
                print(Author)
                PrevAuthor = TLG
            else:
                print(TLG)
        elif soup.treebank.sentence['document_id']:
            docID = soup.treebank.sentence['document_id']

            TLG = docID.split(':')[-1]
            #author = TLG.split('.')[0]
            #work = TLG.split('.')[1]
            if not TLG == PrevAuthor:
                print(TLG)
                PrevAuthor = TLG
            else:
                print(TLG)
        for sentence in soup('sentence'):
            for word in sentence('word'):
                if word.has_attr('postag'):
                    postag = word['postag']
                    #print(postag)
                    if not postag in Except and postag:
                        #print(postag)
                        pos    = postag[0]
                        person = postag[1]
                        number = postag[2]
                        tense  = postag[3]
                        mood   = postag[4]
                        voice  = postag[5]
                        #gender = postag[6]
                        #case   = postag[7]

                        if (pos == 'v'):
                            category = tense + mood + voice
                            verbForms[category] += 1
        print(verbForms)

In [84]:
searchfiles = []
searchfiles = addfiles(GORPATH, searchfiles)
searchfiles = addfiles(PEDPATH, searchfiles)
searchfiles = addfiles(MAMPATH, searchfiles)

for FILENAME in searchfiles:
    VerbTenses(FILENAME)

tlg0026.tlg001.perseus-grc1
defaultdict(<class 'int'>, {'apm': 7, 'apa': 25, 'pia': 119, 'rpa': 30, 'ppa': 66, 'ppe': 36, 'fia': 9, 'aim': 15, 'pna': 110, 'ana': 19, 'aia': 35, 'ria': 18, 'pne': 13, 'fip': 1, 'iia': 22, 'rne': 1, 'fim': 18, 'anm': 10, 'pie': 40, 'rpe': 17, 'rma': 2, 'pse': 11, 'psa': 31, 'asa': 16, 'ame': 5, 'anp': 3, 'fpe': 1, 'rsa': 4, 'aie': 4, 'app': 8, 'fnm': 3, 'pma': 25, 'pme': 6, 'asp': 8, 'pim': 2, 'rie': 4, 'ase': 3, 'ama': 11, 'amp': 1, 'aoa': 5, 'aip': 4, 'fpm': 2, 'asm': 5, 'iie': 10, 'fna': 3, 'ape': 2, 'rnm': 1, 'aom': 2, 'fie': 1, 'lia': 1, 'iim': 1, 'ane': 2})
tlg0026.tlg001.perseus-grc1
defaultdict(<class 'int'>, {'lie': 1, 'ria': 19, 'fia': 23, 'app': 7, 'aim': 15, 'iia': 32, 'rpe': 11, 'aia': 51, 'pia': 110, 'ppa': 60, 'rna': 8, 'apm': 14, 'iie': 5, 'aip': 11, 'rpa': 14, 'ppe': 30, 'apa': 40, 'ana': 19, 'pma': 8, 'aoa': 5, 'psa': 5, 'pne': 14, 'aom': 2, 'ppm': 6, 'fip': 1, 'fpa': 2, 'pna': 70, 'pie': 31, 'asa': 10, 'ama': 8, 'asp': 1, 'ane': 2, 'fnm

tlg0008.tlg001.perseus-grc1
defaultdict(<class 'int'>, {'iie': 9, 'pia': 95, 'ppe': 37, 'pie': 19, 'ana': 12, 'pne': 19, 'psa': 8, 'rsa': 1, 'lie': 2, 'rie': 6, 'rne': 3, 'ria': 6, 'rnm': 1, 'ppa': 71, 'aia': 25, 'aoa': 2, 'rpa': 7, 'apa': 26, 'apm': 8, 'pna': 34, 'app': 10, 'iia': 34, 'aim': 14, 'aip': 7, 'rpe': 7, '___': 7, 'fia': 6, 'asa': 6, 'fim': 5, 'pma': 3, 'amp': 1, 'psm': 1, 'rna': 4, 'ppm': 2, 'asm': 2, 'pse': 3, 'asp': 1, 'anm': 2, 'fpa': 1, 'poa': 1, 'ape': 1, 'anp': 1, 'fpm': 1, 'fom': 1, 'fna': 1, 'aie': 1, 'lia': 1, 'poe': 1, 'fip': 1})
tlg0008.tlg001.perseus-grc1
defaultdict(<class 'int'>, {'apa': 39, 'iia': 22, 'pne': 10, 'pna': 20, 'ppa': 37, 'psa': 4, 'rpa': 2, 'fpp': 1, '___': 10, 'iie': 5, 'pia': 55, 'aia': 41, 'ppe': 15, 'aoa': 4, 'anm': 2, 'anp': 4, 'aim': 12, 'ana': 12, 'fpa': 2, 'aip': 12, 'pie': 7, 'fna': 1, 'rpe': 7, 'rna': 1, 'app': 6, 'fpm': 3, 'apm': 6, 'poa': 2, 'ane': 1, 'ria': 1, 'lia': 1, 'aom': 1, 'asa': 1, 'rne': 1, 'ama': 1, 'pma': 2, 'rie': 2, 'fi

tlg0008.tlg001.perseus-grc1
defaultdict(<class 'int'>, {'pia': 136, 'pna': 38, 'pie': 34, 'ppa': 67, 'ppe': 14, 'aia': 32, 'ana': 18, 'iia': 24, 'ria': 11, 'rie': 3, 'fpm': 4, 'aim': 4, 'asa': 2, 'rpa': 7, 'pne': 5, 'apa': 13, 'aie': 2, 'rna': 2, 'aoa': 1, 'aip': 4, 'aop': 1, 'lia': 1, 'app': 3, 'iie': 3, 'rpe': 11, 'psa': 6, 'pse': 2, '___': 12, 'pnm': 2, 'apm': 2, 'anp': 2, 'asp': 2, 'ppm': 2, 'anm': 2, 'pma': 2, 'ama': 1, 'poe': 1, 'amm': 1, 'fia': 1, 'ase': 1})
tlg0008.tlg001.perseus-grc1
defaultdict(<class 'int'>, {'pia': 112, 'pna': 36, 'pie': 33, 'ppa': 69, 'ppe': 13, 'aip': 5, 'fpa': 1, 'iia': 22, 'ria': 11, 'rie': 3, 'aia': 31, 'fpm': 4, 'aim': 3, 'asa': 2, 'pse': 4, 'rpa': 7, 'ana': 17, 'pne': 5, 'apa': 13, 'aie': 3, 'rna': 2, 'aoa': 1, 'aop': 1, 'lia': 1, 'app': 3, 'iie': 3, 'rpe': 11, 'psa': 5, 'pnm': 2, 'pma': 3, 'apm': 2, 'anp': 2, 'asp': 2, 'ppm': 2, 'anm': 2, 'ama': 1, 'poe': 1, 'amm': 1, 'ase': 1})
tlg0008.tlg001.perseus-grc1
defaultdict(<class 'int'>, {'pia': 83, 'iia

tlg0014.tlg018.perseus-grc1
defaultdict(<class 'int'>, {'pie': 27, 'ppa': 102, 'pia': 83, 'ana': 44, 'anm': 16, 'pna': 61, 'poa': 9, 'rie': 9, 'ane': 1, 'rna': 7, 'ppe': 33, 'pne': 19, 'psa': 10, 'rpe': 31, 'fia': 7, 'ria': 19, 'fip': 2, 'fim': 9, 'apm': 5, 'iie': 32, 'apa': 36, 'fna': 6, 'iia': 67, 'aia': 26, 'aim': 13, 'pse': 7, 'ame': 3, 'ase': 2, 'amm': 2, 'pma': 10, 'rme': 1, 'aip': 5, 'fnm': 2, 'rpa': 14, 'aoa': 9, 'fpa': 1, 'anp': 4, 'lia': 5, 'lie': 5, 'fpp': 1, 'aom': 3, 'aie': 1, 'app': 8, 'pim': 2, 'asp': 3, 'rne': 5, 'poe': 2, 'lim': 1, 'asa': 4, 'rma': 1, 'ape': 1, 'pnm': 1, 'iim': 2, 'aop': 1, 'asm': 1})
tlg0014.tlg018.perseus-grc1
defaultdict(<class 'int'>, {'rpa': 17, 'ppa': 103, 'pne': 13, 'iia': 61, 'fna': 6, 'pna': 70, 'ppe': 32, 'asm': 2, 'aia': 80, 'ria': 17, 'iie': 22, 'pie': 24, 'ana': 40, 'pma': 15, 'rpe': 17, 'app': 13, 'aoa': 5, 'aip': 17, 'anp': 5, 'pia': 89, 'apa': 32, 'psa': 9, 'rna': 2, 'iim': 2, 'aim': 10, 'fip': 2, 'pim': 2, 'rne': 8, 'asa': 5, 'fim': 10

tlg0060.tlg001.perseus-grc3
defaultdict(<class 'int'>, {'rpa': 25, 'iia': 76, 'ppp': 2, 'aia': 164, 'ppa': 95, 'apa': 104, 'pna': 65, 'rpe': 47, 'apm': 44, 'ppe': 59, 'app': 51, 'aim': 30, 'pne': 10, 'fia': 9, 'ana': 40, 'asa': 9, 'fpm': 1, 'iie': 30, 'aip': 31, 'pie': 2, 'pia': 11, 'rna': 5, 'anm': 13, 'anp': 10, 'fim': 7, 'ane': 3, 'ria': 4, 'fna': 16, 'rne': 1, 'aie': 2, 'lia': 2, 'fpa': 7, 'asm': 3, 'pse': 3, 'ape': 5, 'iim': 2, 'ppm': 2, 'ama': 1, 'fnm': 5, 'lie': 1, 'psa': 2, 'aop': 1, 'fpe': 1})
tlg0060.tlg001.perseus-grc3
defaultdict(<class 'int'>, {'rpa': 8, 'iia': 67, 'aip': 20, 'aia': 154, 'ppe': 71, 'aim': 32, 'ppa': 139, 'pna': 58, 'anm': 13, 'apm': 35, 'fnm': 5, 'lie': 4, 'fna': 21, 'rpe': 27, 'fnp': 3, 'ape': 3, 'fpa': 7, 'pie': 3, 'asa': 9, 'app': 35, 'apa': 92, 'ria': 5, 'fpm': 3, 'psa': 4, 'pne': 10, 'ana': 51, 'iie': 14, 'lia': 4, 'fia': 3, 'anp': 2, 'rne': 4, 'fim': 1, 'aoa': 2, 'aie': 2, 'rna': 3, 'ane': 3, 'asm': 3, 'pia': 7, 'fip': 1, 'poa': 1})
tlg0060.tlg001.pe

...

tlg0059.tlg002.perseus-grc1
defaultdict(<class 'int'>, {'ria': 76, 'aim': 33, 'iia': 106, 'ana': 56, 'aia': 63, 'pne': 50, 'asp': 2, 'ppa': 217, 'pna': 250, 'anp': 2, 'fip': 3, 'pse': 9, 'pia': 462, 'poa': 52, 'fim': 43, 'rpe': 9, 'ppe': 77, 'apa': 33, 'ama': 12, 'pie': 114, 'psa': 24, 'rpa': 27, 'lie': 1, 'anm': 17, 'pim': 7, 'rna': 29, 'aip': 10, 'poe': 7, 'pma': 20, 'asa': 28, 'iie': 18, 'fna': 10, 'rne': 8, 'aoa': 27, 'rma': 7, 'fia': 32, 'ane': 3, 'ame': 2, 'fpa': 4, 'ppm': 3, 'iim': 2, 'aom': 5, 'fpm': 5, 'lia': 10, 'foa': 1, 'apm': 15, 'amm': 6, '-sa': 3, '___': 1, 'pme': 5, 'rme': 1, 'pnm': 4, 'pom': 4, 'pp-': 5, 'roa': 1, 'app': 5, 'fom': 2, 'asm': 6, 'ape': 11, 'fnp': 1, 'rsa': 1, 'aie': 4, 'rie': 7, 'aop': 4, 'fnm': 5, 'fie': 2, 'fne': 1, 'aoe': 2})
1.1
defaultdict(<class 'int'>, {'pna': 75, 'pia': 53, 'iia': 109, 'rpa': 10, 'aia': 89, 'ppe': 66, 'ppa': 156, 'pie': 16, 'apm': 22, 'ria': 10, 'apa': 71, 'ana': 51, 'rie': 6, 'aoa': 4, 'aim': 12, 'anp': 4, 'pne': 17, 'pma': 2, '

tlg0543.tlg001.perseus-grc1
defaultdict(<class 'int'>, {'apm': 25, 'aia': 60, 'pna': 33, 'rpe': 17, 'rpa': 19, 'apa': 63, 'aim': 8, 'ppe': 40, 'pne': 10, 'aip': 4, 'app': 10, 'aie': 1, 'fpm': 4, 'ppa': 59, 'iia': 34, 'ana': 38, 'anm': 6, 'iie': 9, 'asa': 2, 'rna': 8, 'rne': 1, 'pia': 29, 'psa': 4, 'ape': 2, 'rpm': 1, 'poe': 2, 'anp': 7, 'pie': 8, 'poa': 1, 'ria': 3, 'fim': 1, 'rme': 2, 'pme': 1, 'pim': 3, 'asm': 1, 'pse': 1, 'fna': 1, 'ase': 1, 'aoa': 2, 'rie': 1})
tlg0543.tlg001.perseus-grc1
defaultdict(<class 'int'>, {'aim': 18, 'pna': 34, 'ppa': 61, 'pne': 26, 'apa': 45, 'ppe': 31, 'pia': 20, 'aia': 50, 'apm': 19, 'ana': 19, 'aoa': 10, 'anp': 5, 'iia': 42, 'app': 11, 'ria': 6, 'aom': 2, 'lie': 2, 'pie': 1, 'rpa': 14, 'aop': 1, 'anm': 7, 'rpe': 12, 'lia': 2, 'ane': 1, 'rna': 2, 'rie': 2, 'fim': 1, 'asa': 2, 'poa': 2, 'ape': 1, 'iim': 1, 'fnm': 1, 'fpm': 2, 'iie': 8, 'asp': 1, 'aip': 3, 'rne': 1, 'asm': 1})
tlg0543.tlg001.perseus-grc1
defaultdict(<class 'int'>, {'aia': 30, 'anm': 7, '

defaultdict(<class 'int'>, {'ppa': 92, 'rpe': 20, 'pna': 25, 'ppe': 36, 'aia': 37, 'pne': 10, 'iia': 53, 'iie': 10, 'apa': 55, 'anp': 3, 'apm': 12, 'app': 16, 'poe': 2, 'poa': 2, 'pia': 24, 'rne': 2, 'rpa': 15, 'lie': 3, 'aip': 7, 'psa': 1, 'ana': 18, 'anm': 5, 'aim': 5, 'fna': 5, 'fpm': 2, 'amm': 1, 'pie': 5, 'rie': 1, 'fpa': 1, 'pse': 1, 'aom': 2, 'aoa': 1, 'asa': 2, 'ape': 1, 'ase': 1, 'ria': 1, 'fim': 1})
tlg0543.tlg001.perseus-grc1
defaultdict(<class 'int'>, {'apa': 40, 'rna': 5, 'pne': 16, 'iia': 63, 'ppe': 42, 'pna': 45, 'aia': 40, 'app': 16, 'fpm': 6, 'anp': 2, 'ane': 2, 'ana': 28, 'ape': 2, 'rpe': 20, 'ppa': 67, 'pie': 5, 'pse': 2, 'asm': 4, 'asa': 5, 'fie': 1, 'rpa': 19, 'iie': 14, 'apm': 17, 'lia': 5, 'lie': 3, 'anm': 7, 'aip': 1, 'aoa': 8, 'psa': 2, 'aim': 7, 'fpe': 1, 'ria': 2, '___': 2, 'rne': 2, 'aop': 4, 'pia': 10, 'fim': 3, 'rie': 1, 'fne': 1, 'ppp': 1, 'asp': 3, 'ppm': 1, 'fpa': 3, 'pnm': 1, 'poe': 1, 'poa': 1, 'fna': 2, 'pp-': 1, 'pma': 2, 'aom': 1})
tlg0543.tlg001.p

defaultdict(<class 'int'>, {'apa': 22, 'apm': 6, 'iie': 15, 'iia': 125, 'aim': 9, 'pie': 46, 'anm': 12, 'app': 6, 'ppe': 45, 'ppa': 124, 'pia': 125, 'poa': 39, 'poe': 22, 'pna': 188, 'aom': 11, 'psa': 30, 'rie': 7, 'roe': 1, 'pma': 8, 'pne': 84, 'fpm': 3, 'fnm': 4, 'pse': 12, 'rna': 8, 'ria': 11, 'rpa': 8, 'ane': 3, 'fia': 9, 'fim': 13, 'asa': 13, 'ana': 12, 'fpe': 1, 'pim': 1, 'ppm': 1, 'tim': 1, 'fna': 10, 'asm': 7, 'ase': 1, 'rme': 1, 'pme': 2, 'rma': 4, 'rpe': 15, 'fpa': 3, 'aia': 23, 'aoa': 13, 'aip': 3, 'aie': 1, 'amp': 1, 'fie': 1, 'roa': 2, 'anp': 2, 'aop': 3, 'pp-': 1, 'asp': 1, 'psm': 1, 'lie': 3, 'lia': 2, 'pom': 1, 'ama': 1, 'ape': 1})
0032-007
defaultdict(<class 'int'>, {'apm': 27, 'iia': 164, 'aia': 114, 'ana': 24, 'ppa': 114, 'rpa': 13, 'ppe': 45, 'apa': 67, 'pna': 85, 'rpe': 18, 'lim': 2, 'pne': 26, 'aim': 37, 'fim': 28, 'iie': 41, 'pia': 103, 'lia': 2, 'poe': 11, '___': 15, 'pie': 36, 'asm': 7, 'pse': 10, 'app': 6, 'pme': 7, 'fia': 14, 'psa': 10, 'aoa': 10, 'ane': 1, '

UnboundLocalError: local variable 'TLG' referenced before assignment

None
